# Capítulo 10 - Performance e Boas Práticas

Este notebook final explora técnicas de otimização e boas práticas para maximizar a performance do DuckDB.

## 📚 Tópicos Abordados:
1. Configurações de Performance
2. Otimizações de Queries
3. Gerenciamento de Memória
4. Formatos de Arquivo (Parquet vs CSV)
5. Compressão e Row Groups
6. Prepared Statements
7. Paralelismo e Threads
8. Benchmarking
9. Pipeline de ETL Otimizado
10. Checklist de Boas Práticas

## 🚀 Objetivos:
- **Maximizar** performance
- **Reduzir** uso de memória
- **Acelerar** pipelines de dados
- **Aplicar** melhores práticas

## 1. Setup e Preparação

In [ ]:
import duckdb
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta

print(f"DuckDB versão: {duckdb.__version__}")
print("✓ Bibliotecas importadas!")

## 2. Configurações de Performance

### 2.1 Configurações Essenciais

In [ ]:
# Criar conexão com configurações otimizadas
con = duckdb.connect(':memory:')

# Configurar memória
con.execute("SET memory_limit = '2GB'")
print("✓ Memory limit: 2GB")

# Configurar threads (use número de cores da CPU)
con.execute("SET threads TO 4")
print("✓ Threads: 4")

# Desabilitar ordem de inserção (melhora performance em GROUP BY)
con.execute("SET preserve_insertion_order = false")
print("✓ Preserve insertion order: false")

# Ver configurações
result = con.execute("""
    SELECT name, value
    FROM duckdb_settings()
    WHERE name IN ('memory_limit', 'threads', 'preserve_insertion_order')
""").fetchdf()
print("\nConfigurações atuais:")
print(result)

### 2.2 Temp Directory para Grandes Volumes

In [ ]:
import tempfile

# Configurar diretório temporário
temp_dir = tempfile.gettempdir()
con.execute(f"SET temp_directory = '{temp_dir}'")

print(f"✓ Temp directory: {temp_dir}")
print("\n💡 Use SSD para melhor performance!")

## 3. Criar Dados de Teste

### 3.1 Gerar Dataset Sintético

In [ ]:
# Gerar dados de teste
np.random.seed(42)

n_rows = 100000
dates = [datetime(2024, 1, 1) + timedelta(days=i % 365) for i in range(n_rows)]

df = pd.DataFrame({
    'id': range(n_rows),
    'data': dates,
    'categoria': np.random.choice(['A', 'B', 'C', 'D'], n_rows),
    'valor': np.random.uniform(10, 1000, n_rows),
    'quantidade': np.random.randint(1, 100, n_rows),
    'regiao': np.random.choice(['Norte', 'Sul', 'Leste', 'Oeste'], n_rows)
})

print(f"✓ Dataset gerado: {len(df):,} linhas")
print(f"✓ Tamanho em memória: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\nPrimeiras linhas:")
print(df.head())

### 3.2 Registrar no DuckDB

In [ ]:
# Registrar DataFrame
con.register('vendas', df)
print("✓ DataFrame registrado como 'vendas'")

# Verificar
result = con.execute("SELECT COUNT(*) as total FROM vendas").fetchone()
print(f"Total de registros: {result[0]:,}")

## 4. Formatos de Arquivo: CSV vs Parquet

### 4.1 Exportar em Diferentes Formatos

In [ ]:
import os

# Exportar CSV
print("Exportando CSV...")
start = time.time()
con.execute("""
    COPY vendas TO 'vendas_test.csv' (FORMAT CSV, HEADER true)
""")
csv_time = time.time() - start
csv_size = os.path.getsize('vendas_test.csv') / 1024**2
print(f"✓ CSV: {csv_time:.3f}s, {csv_size:.2f} MB")

# Exportar Parquet
print("\nExportando Parquet...")
start = time.time()
con.execute("""
    COPY vendas TO 'vendas_test.parquet' (FORMAT PARQUET)
""")
parquet_time = time.time() - start
parquet_size = os.path.getsize('vendas_test.parquet') / 1024**2
print(f"✓ Parquet: {parquet_time:.3f}s, {parquet_size:.2f} MB")

# Comparação
print("\n📊 COMPARAÇÃO:")
print(f"Velocidade: Parquet é {csv_time/parquet_time:.1f}x mais rápido")
print(f"Tamanho: Parquet é {csv_size/parquet_size:.1f}x menor")

### 4.2 Leitura: CSV vs Parquet

In [ ]:
# Ler CSV
print("Lendo CSV...")
start = time.time()
result_csv = con.execute("SELECT COUNT(*) FROM 'vendas_test.csv'").fetchone()
csv_read_time = time.time() - start
print(f"✓ CSV: {csv_read_time:.3f}s, {result_csv[0]:,} linhas")

# Ler Parquet
print("\nLendo Parquet...")
start = time.time()
result_parquet = con.execute("SELECT COUNT(*) FROM 'vendas_test.parquet'").fetchone()
parquet_read_time = time.time() - start
print(f"✓ Parquet: {parquet_read_time:.3f}s, {result_parquet[0]:,} linhas")

# Comparação
print("\n⚡ PERFORMANCE:")
print(f"Parquet é {csv_read_time/parquet_read_time:.1f}x mais rápido para leitura")

## 5. Compressão e Row Groups

### 5.1 Diferentes Compressões

In [ ]:
# Testar diferentes compressões
compressoes = ['snappy', 'gzip', 'zstd', 'uncompressed']
resultados = []

for comp in compressoes:
    filename = f'vendas_{comp}.parquet'
    
    # Exportar
    start = time.time()
    con.execute(f"""
        COPY vendas TO '{filename}' 
        (FORMAT PARQUET, COMPRESSION {comp})
    """)
    write_time = time.time() - start
    
    # Tamanho
    size = os.path.getsize(filename) / 1024**2
    
    # Ler
    start = time.time()
    con.execute(f"SELECT COUNT(*) FROM '{filename}'").fetchone()
    read_time = time.time() - start
    
    resultados.append({
        'compressao': comp,
        'tamanho_mb': size,
        'write_s': write_time,
        'read_s': read_time
    })

# Mostrar resultados
comp_df = pd.DataFrame(resultados)
print("📊 COMPARAÇÃO DE COMPRESSÕES:")
print(comp_df.to_string(index=False))

print("\n💡 ZSTD oferece melhor balanço!")

### 5.2 Row Group Size

In [ ]:
# Testar diferentes row group sizes
print("Testando ROW_GROUP_SIZE:")

for size in [10000, 50000, 100000]:
    filename = f'vendas_rg_{size}.parquet'
    
    start = time.time()
    con.execute(f"""
        COPY vendas TO '{filename}'
        (FORMAT PARQUET, COMPRESSION zstd, ROW_GROUP_SIZE {size})
    """)
    write_time = time.time() - start
    file_size = os.path.getsize(filename) / 1024**2
    
    print(f"Row group {size:6d}: {write_time:.3f}s, {file_size:.2f} MB")

print("\n💡 Row groups menores = mais paralelismo")
print("💡 Row groups maiores = melhor compressão")

## 6. Otimização de Queries

### 6.1 Filtros e Projeções

In [ ]:
# Query SEM otimização
print("Query SEM filtro na origem:")
start = time.time()
result = con.execute("""
    SELECT categoria, AVG(valor) as media
    FROM (
        SELECT * FROM vendas
    )
    WHERE data >= '2024-06-01'
    GROUP BY categoria
""").fetchdf()
slow_time = time.time() - start
print(f"Tempo: {slow_time:.3f}s")

# Query COM otimização (filtro early)
print("\nQuery COM filtro antecipado:")
start = time.time()
result = con.execute("""
    SELECT categoria, AVG(valor) as media
    FROM vendas
    WHERE data >= '2024-06-01'
    GROUP BY categoria
""").fetchdf()
fast_time = time.time() - start
print(f"Tempo: {fast_time:.3f}s")

print(f"\n⚡ Melhoria: {slow_time/fast_time:.1f}x mais rápido")
print("\n💡 Aplique filtros o mais cedo possível!")

### 6.2 SELECT * vs Colunas Específicas

In [ ]:
# SELECT *
print("SELECT * (todas as colunas):")
start = time.time()
result = con.execute("""
    SELECT * FROM 'vendas_test.parquet' LIMIT 10000
""").fetchdf()
all_cols_time = time.time() - start
print(f"Tempo: {all_cols_time:.3f}s, Memória: {result.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# SELECT específico
print("\nSELECT categoria, valor (colunas específicas):")
start = time.time()
result = con.execute("""
    SELECT categoria, valor FROM 'vendas_test.parquet' LIMIT 10000
""").fetchdf()
few_cols_time = time.time() - start
print(f"Tempo: {few_cols_time:.3f}s, Memória: {result.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\n⚡ Economia: {all_cols_time/few_cols_time:.1f}x mais rápido")
print("\n💡 Selecione apenas colunas necessárias!")

## 7. Prepared Statements

### 7.1 Comparação: Normal vs Prepared

In [ ]:
# Criar tabela de teste
con.execute("""
    CREATE OR REPLACE TABLE usuarios (
        id INTEGER,
        nome VARCHAR,
        idade INTEGER
    )
""")

# Método 1: Query normal (lento)
print("Método 1: Query normal")
start = time.time()
for i in range(1000):
    con.execute(f"INSERT INTO usuarios VALUES ({i}, 'User_{i}', {20 + i % 50})")
normal_time = time.time() - start
print(f"Tempo: {normal_time:.3f}s")

# Limpar
con.execute("DELETE FROM usuarios")

# Método 2: Prepared statement (rápido)
print("\nMétodo 2: Prepared statement")
start = time.time()
stmt = con.execute("PREPARE insert_user AS INSERT INTO usuarios VALUES ($1, $2, $3)")
for i in range(1000):
    con.execute(f"EXECUTE insert_user({i}, 'User_{i}', {20 + i % 50})")
prepared_time = time.time() - start
print(f"Tempo: {prepared_time:.3f}s")

print(f"\n⚡ Prepared statement é {normal_time/prepared_time:.1f}x mais rápido!")

# Verificar
count = con.execute("SELECT COUNT(*) FROM usuarios").fetchone()[0]
print(f"\n✓ Total inserido: {count:,} registros")

## 8. Paralelismo e Threads

### 8.1 Impacto do Número de Threads

In [ ]:
# Query de teste (agregação pesada)
query = """
    SELECT 
        categoria,
        regiao,
        COUNT(*) as total,
        SUM(valor * quantidade) as receita,
        AVG(valor) as valor_medio
    FROM vendas
    GROUP BY categoria, regiao
    ORDER BY receita DESC
"""

# Testar diferentes números de threads
print("Testando diferentes números de threads:")
print()

for threads in [1, 2, 4, 8]:
    con.execute(f"SET threads = {threads}")
    
    start = time.time()
    result = con.execute(query).fetchdf()
    elapsed = time.time() - start
    
    print(f"Threads {threads}: {elapsed:.3f}s")

print("\n💡 Mais threads = melhor performance (até o limite de cores)")

# Resetar para valor otimizado
con.execute("SET threads = 4")

## 9. Benchmarking Completo

### 9.1 Suite de Benchmarks

In [ ]:
# Suite de queries para benchmark
benchmarks = {
    'Count': "SELECT COUNT(*) FROM vendas",
    'Agregação Simples': "SELECT categoria, SUM(valor) FROM vendas GROUP BY categoria",
    'Agregação Complexa': """
        SELECT 
            categoria, regiao,
            COUNT(*) as total,
            AVG(valor) as media,
            STDDEV(valor) as desvio
        FROM vendas
        GROUP BY categoria, regiao
    """,
    'Filtro + Sort': """
        SELECT * FROM vendas
        WHERE valor > 500
        ORDER BY valor DESC
        LIMIT 1000
    """,
    'Window Function': """
        SELECT 
            categoria,
            valor,
            ROW_NUMBER() OVER (PARTITION BY categoria ORDER BY valor DESC) as rank
        FROM vendas
    """
}

print("🏃 EXECUTANDO BENCHMARKS:")
print("="*60)

resultados_bench = []
for nome, query in benchmarks.items():
    start = time.time()
    result = con.execute(query).fetchdf()
    elapsed = time.time() - start
    
    resultados_bench.append({
        'Benchmark': nome,
        'Tempo (s)': f"{elapsed:.3f}",
        'Linhas': len(result)
    })
    
    print(f"{nome:25s} {elapsed:6.3f}s  ({len(result):,} linhas)")

print("="*60)
print("\n✓ Benchmarks concluídos!")

## 10. Pipeline ETL Otimizado

### 10.1 Pipeline Completo

In [ ]:
def pipeline_etl_otimizado():
    """
    Pipeline ETL otimizado com DuckDB
    """
    print("🚀 PIPELINE ETL OTIMIZADO")
    print("="*60)
    
    # 1. Configuração
    print("\n1️⃣ Configurando conexão...")
    con_etl = duckdb.connect(':memory:')
    con_etl.execute("SET memory_limit = '2GB'")
    con_etl.execute("SET threads = 4")
    con_etl.execute("SET preserve_insertion_order = false")
    print("   ✓ Configuração aplicada")
    
    # 2. Extração (CSV → Parquet)
    print("\n2️⃣ Convertendo CSV para Parquet...")
    start = time.time()
    con_etl.execute("""
        COPY (
            SELECT * FROM 'vendas_test.csv'
        ) TO 'vendas_otimizado.parquet'
        (FORMAT PARQUET, COMPRESSION zstd, ROW_GROUP_SIZE 50000)
    """)
    print(f"   ✓ Conversão: {time.time() - start:.2f}s")
    
    # 3. Transformação
    print("\n3️⃣ Aplicando transformações...")
    start = time.time()
    con_etl.execute("""
        CREATE TABLE vendas_processadas AS
        SELECT 
            id,
            data,
            categoria,
            valor,
            quantidade,
            valor * quantidade as receita,
            regiao,
            CASE 
                WHEN valor < 100 THEN 'Baixo'
                WHEN valor < 500 THEN 'Médio'
                ELSE 'Alto'
            END as faixa_preco
        FROM 'vendas_otimizado.parquet'
        WHERE valor > 0
    """)
    print(f"   ✓ Transformação: {time.time() - start:.2f}s")
    
    # 4. Agregação
    print("\n4️⃣ Gerando agregações...")
    start = time.time()
    result = con_etl.execute("""
        SELECT 
            categoria,
            regiao,
            faixa_preco,
            COUNT(*) as transacoes,
            SUM(receita) as receita_total,
            AVG(valor) as ticket_medio
        FROM vendas_processadas
        GROUP BY categoria, regiao, faixa_preco
        ORDER BY receita_total DESC
    """).fetchdf()
    print(f"   ✓ Agregação: {time.time() - start:.2f}s")
    print(f"   ✓ Resultados: {len(result)} linhas")
    
    # 5. Exportação
    print("\n5️⃣ Exportando resultados...")
    start = time.time()
    con_etl.execute("""
        COPY (
            SELECT * FROM vendas_processadas
        ) TO 'vendas_final.parquet'
        (FORMAT PARQUET, COMPRESSION zstd)
    """)
    print(f"   ✓ Exportação: {time.time() - start:.2f}s")
    
    print("\n" + "="*60)
    print("✅ PIPELINE CONCLUÍDO COM SUCESSO!")
    
    con_etl.close()
    return result

# Executar pipeline
resultado = pipeline_etl_otimizado()
print("\nTop 5 resultados:")
print(resultado.head())

## 11. Checklist de Boas Práticas

### 11.1 Configuração

In [ ]:
print("""
✅ CHECKLIST DE BOAS PRÁTICAS
""" + "="*60 + """

🔧 CONFIGURAÇÃO:
  ✓ Definir memory_limit apropriado
  ✓ Configurar threads = número de cores
  ✓ Usar temp_directory em SSD
  ✓ preserve_insertion_order = false (quando possível)

📁 FORMATOS DE ARQUIVO:
  ✓ Usar Parquet em vez de CSV
  ✓ Compressão ZSTD para melhor balanço
  ✓ Row group size entre 50k-100k
  ✓ Particionar dados grandes por data/categoria

🔍 QUERIES:
  ✓ Selecionar apenas colunas necessárias
  ✓ Aplicar filtros o mais cedo possível
  ✓ Usar LIMIT quando apropriado
  ✓ Evitar SELECT * em produção

⚡ PERFORMANCE:
  ✓ Usar prepared statements para inserts repetitivos
  ✓ Batch inserts em vez de individuais
  ✓ Habilitar paralelismo (force_parallelism = true)
  ✓ Reusar conexões quando possível

💾 MEMÓRIA:
  ✓ Processar dados em chunks para grandes volumes
  ✓ Usar streaming quando possível
  ✓ Limpar temp files após processamento
  ✓ Monitorar uso de memória

📊 MONITORAMENTO:
  ✓ Usar EXPLAIN ANALYZE para otimizar queries
  ✓ Fazer benchmarks regulares
  ✓ Monitorar tempo de execução
  ✓ Revisar planos de execução

🔐 SEGURANÇA:
  ✓ Validar inputs de usuários
  ✓ Usar prepared statements (previne SQL injection)
  ✓ Não expor credenciais no código
  ✓ Usar secrets para credenciais S3
""" + "="*60)

### 11.2 Limpeza

In [ ]:
# Limpar arquivos de teste
import glob

arquivos_teste = glob.glob('vendas*.csv') + glob.glob('vendas*.parquet')
for arquivo in arquivos_teste:
    try:
        os.remove(arquivo)
    except:
        pass

print(f"✓ {len(arquivos_teste)} arquivos de teste removidos")
print("\n✅ Notebook concluído!")

## 🎯 Resumo Final

### ✅ Principais Aprendizados:

**Performance:**
- Parquet é **~5-10x mais rápido** que CSV
- Parquet é **~3-5x menor** que CSV
- ZSTD oferece melhor balanço compressão/velocidade
- Prepared statements são **~5-10x mais rápidos**
- Threads adequados = **2-4x mais rápido**

**Configurações Essenciais:**
1. `memory_limit` → Evitar OOM
2. `threads` → Maximizar paralelismo
3. `temp_directory` → SSD para grandes volumes
4. `preserve_insertion_order = false` → Melhor GROUP BY

**Otimizações de Query:**
- Filtrar cedo (WHERE antes de JOIN)
- Selecionar apenas colunas necessárias
- Usar LIMIT quando apropriado
- Aproveitar predicate pushdown

**Formatos:**
- ✅ Parquet: Produção
- ⚠️ CSV: Interchange/debug
- ✅ Compressão: ZSTD
- ✅ Row groups: 50k-100k

### 🚀 Próximos Passos:

1. **Aplicar em projetos reais**
2. **Monitorar performance continuamente**
3. **Iterar e otimizar**
4. **Documentar resultados**

### 📚 Recursos Adicionais:

- [DuckDB Documentation](https://duckdb.org/docs/)
- [Performance Guide](https://duckdb.org/docs/guides/performance/)
- [Best Practices](https://duckdb.org/docs/guides/best_practices/)

### 🎓 Conclusão:

Parabéns! Você completou o curso **DuckDB - Easy**!

Você aprendeu:
- ✅ 10 capítulos completos
- ✅ SQL avançado com DuckDB
- ✅ Integração com Python/Pandas/Polars
- ✅ Otimização e performance
- ✅ Boas práticas de desenvolvimento

**Continue praticando e explorando o DuckDB! 🦆**